In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython.display import display, HTML
import matplotlib.colors
import networkx as nx

import matplotlib.ticker as mtick

path_data = "../data/omn/"
# file that has before and after shocks
file_arch = 'occ_archetypes_thresholds_relbase_2034_2038.csv'
network_type = 'cc' #'uniform' #'baselinemerge' # 'tasksim' 'cc' # baseline is merge weight
file_edgelist = 'edgelist_cc_mobility_merge.csv' # 'edgelist_cc_mobility_merge_joris.csv'
path_fig = "../results/figs/"
annotate_plot_bool = False # True
start_year = 2021
mark_year = 2034 # year marking the period split. Exclusive of mark year
# for first period, inclusive for latter period. 
end_year = 2038
background_bool = True
circle_selected_bool = True
circle_occs = ['Power Plant Operators, Distributors, and Dispatchers',
               'Dredge, Excavating, and Loading Machine Operators',
               'Construction Laborers',
               'Wind Turbine Service Technicians']
circle_occs_code = ['A', 'B', 'C', 'D']

neigh_in_bef  = 'pool_shock_inbefore' # 'frac_updown_inbefore' # 'neigh_shock_inbefore_norm'
neigh_in_af   = 'pool_shock_inafter' # 'frac_updown_inafter' # 'neigh_shock_inafter_norm'
neigh_out_bef ='pool_shock_outbefore' #  'frac_updown_outbefore' # 'neigh_shock_outbefore_norm'
neigh_out_af  ='pool_shock_outafter' #  'frac_updown_outafter' # 'neigh_shock_outafter_norm'


fig_out_name = "assortativity_symlog_" + network_type + "_" + str(mark_year) \
    + "_" + str(end_year) 
    
    
fig_out_name_after = "assortativity_symlog_" + network_type + "_" + str(mark_year) \
    + "_" + str(end_year) + '_after'

period_before = mark_year - start_year
period_after = end_year - mark_year

file_assort = 'occ_shock_network' + network_type + '_assortativity_flows.csv'
df_occ = pd.read_csv(path_data + file_assort)

# edgelist
df_edgelist =  pd.read_csv(path_data + file_edgelist)
df_nodes =  pd.read_csv(path_data + file_arch)

shock_after_name = 'shock_after_' + str(mark_year)
shock_before_name = 'shock_before_' + str(mark_year)

bool_perm = df_occ['Permanent_boost_r0.01'] == 1
bool_phaseout = df_occ['Phase_out_r0.01'] == 1
bool_temp = df_occ['Temporary_boost_r0.01'] == 1
# bool_decrease_big = 

###########
# useful functions
###########


df_occ.columns

def selected_occ(df, shock=shock_before_name + '_norm', \
    neighbors_in=neigh_in_bef, \
    neighbors_out=neigh_out_bef, shock_bool ='pos',\
        criteria_shock=0, criteria_neigh_pos=0, criteria_neigh_neg=0):
    # selecting archetypes
    if shock_bool == 'pos':
        df_selected = df[(df[shock] > criteria_shock) | \
            ( df[neighbors_in] > criteria_neigh_pos) | \
               (df[neighbors_out] < criteria_neigh_neg) ]
         
                    # df[((df[shock] > criteria_shock) | \
                    # (df[neighbors_in] > criteria_neigh_pos)) |\
                    
                    # ((df[shock] > criteria_shock) | \
                    # (df[neighbors_out] < criteria_neigh_neg)) ]
         
    elif shock_bool == 'neg':
        df_selected = df[((df[shock] < criteria_shock) | \
                    (df[neighbors_in] > criteria_neigh_pos)) |\
                    
                    ((df[shock] < criteria_shock) | \
                    (df[neighbors_out] < criteria_neigh_neg)) ]
   
    # shortening titles

    df_selected["short_title"] = df_selected["OCC_TITLE"]\
        .str.replace('First-Line Supervisors of', 'Supervisors of\n')
    df_selected["short_title"] = df_selected["short_title"]\
        .str.replace('Miscellaneous', 'Misc.')
    df_selected["short_title"] = df_selected["short_title"]\
        .str.replace('and', '&')
    df_selected["short_title"] = df_selected["short_title"]\
        .str.replace(',', ',\n')
        
    return df_selected


def annotate_plot(sc, ax, df_selected, scale='symlog'):
    indexes = df_selected.index.tolist()
    
    for i in indexes:
        txt = df_selected["short_title"].loc[i]
        x, y =  sc.get_offsets().data[i]
        ax.annotate(txt, (x, y), horizontalalignment='right',\
            verticalalignment='top')


def background_color(sc, ax, xlim, ylim, vertical=True, identity=True):

    if identity == True:
        if vertical == True:
            gradient_fill_vertical(np.array(xlim), np.array(ylim), fill_color = 'grey', ax=ax, through_zero = identity)
        else:
            a = -1*min(abs(xlim[0]), abs(ylim[1]))
            if a > xlim[0]:
                x = np.array([xlim[0], a, 0, 0.00000001])
                y = np.array([-1*a, -1*a, 0, ylim[0]])
            else:
                x = np.array([a, 0, 0.000001])
                y = np.array([-1*a, 0, ylim[0]])
            #ax.plot(x, y, linewidth = 1.5, c='black', alpha=0.7)
            #ax.axvline(x=0, linewidth = 1.5, c='black', alpha=0.7)
            gradient_fill(x,y, fill_color = 'grey', ax=ax)
        
        # other side

    if vertical == True:
        gradient_fill_vertical(np.array(xlim), np.array(ylim), fill_color = 'grey', ax=ax, direction='up', through_zero = identity)
    else:
        a = min(abs(xlim[1]), abs(ylim[0]))
        if a < xlim[1]:
            x = np.array([0, 0.0000001, a, xlim[1]])
            y = np.array([ylim[1], 0, -1*a, -1*a])
        else:
            x = np.array([0, 0.0000001, a])
            y = np.array([ylim[1], 0, -1*a])
        gradient_fill(x,y, fill_color = 'grey', ax=ax, direction='up')


    #add identity line
    if identity == True:
        minval = max(xlim[0], ylim[0])
        maxval = min(xlim[1], ylim[1])
        ax.plot([minval, maxval], [minval, maxval], linewidth = 0.9, c='grey', alpha=0.7, zorder=0, linestyle='--')
        
def background_text(sc, ax, rotate, identity = 'down', zoombox=False):
    if (zoombox == True) & (identity == 'down'):
        ax.annotate('Identity line: \nassortative frictions', (0.0105, 0.017), rotation=-rotate)
    elif identity == 'down':
        ax.annotate('Workers struggle \nfinding jobs', (-0.19, -0.0043))
        ax.annotate('Employers can find \nworkers easily', (0.67, -0.0043))
        ax.annotate('Workers can find \njobs in their \nnetwork', (-0.19, 0.026))
        ax.annotate('Employers struggle \nfinding workers', (0.67, 0.009))
        ax.annotate('Identity line: \nassortative frictions', (-0.016, 0.021), rotation=-rotate)
    elif zoombox == True:
        ax.annotate('Identity line: \nassortative frictions', (-0.0046, -0.0036), rotation=-rotate)
    else:
        ax.annotate('Workers struggle \nfinding jobs', (-0.047, -0.029))
        ax.annotate('Employers can find \nworkers easily', (0.15, -0.029))
        ax.annotate('Workers can find jobs \nin their network', (-0.047, 0.002))
        ax.annotate('Employers struggle \nfinding workers', (0.15, 0.002))
        ax.annotate('Identity line: \nassortative frictions', (-0.0055, -0.0015), rotation=-rotate)

def vertical_line_with_text(sc, ax, identity='down'):
    ax.axvline(x=0, color='grey', alpha=0.6, linewidth=0.5)

    
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon

def gradient_fill_vertical(x, y, fill_color=None, ax=None, direction='down', through_zero = True, **kwargs):
    if ax is None:
        ax = plt.gca()

    zorder = 0
    #alpha = line.get_alpha()
    alpha = 0.3

    z = np.empty((100, 1, 4), dtype=float)
    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:,:,:3] = rgb
    if direction == 'down':
        z[:,:,-1] = np.linspace(alpha, 0,100)[:,None]
    else: # direction == 'up':
        z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]

    xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    if direction == 'down':
        xmax = 0
        ymax = 0.0075
    else:
        if through_zero == True:
            xmin = 0
            ymin = -0.0075
        else:
            ymin = -0.0075
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    ax.autoscale(True)
    return None, im



def gradient_fill(x, y, fill_color=None, ax=None, direction='down', **kwargs):
    # this function is from https://stackoverflow.com/questions/29321835/is-it-possible-to-get-color-gradients-under-curve-in-matplotlib
    # and modified to fit my needs 
    # x, y are arrays of the same length
    # fill_color is the color of the filled area
    # direction is 'down' (default) or 'up'
    # kwargs passes keyword arguments to plt.plot

    if ax is None:
        ax = plt.gca()
            
    
    xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()

    zorder = 0.5
    lim = [-0.05, 0.05]
    xlim = [-0.1, 0.15]
    ylim = [-0.03, 0.03]
    alphalim=0.3
    xmaxlim = 0.15

    line, = ax.plot(x, y, **kwargs, zorder = zorder, c = fill_color)
    if fill_color is None:
        fill_color = line.get_color()

    alpha = 0.4
    #alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 100, 4), dtype=float)
    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:,:,:3] = rgb
    if direction == 'down':
        z[:,:,-1] = np.linspace(alpha, 0.0,100)[:,None]

        z[:, :, -1] = z[:, :, -1] + z[:, :, -1].T * (xlim[0] / lim[0])
        z[:, :, -1] -= 0.05
        z[:,:,-1] = z[:,:,-1] / z[:,:,-1].max() * alpha
    else: # direction == 'up':
        z[:,:,-1] = np.linspace(0.0, alpha, 100)[:,None]

        z[:, :, -1] = z[:, :, -1] + z[:, :, -1].T * (xlim[1] / lim[1])
        z[:, :, -1] -= 0.05
        z[:,:,-1] = z[:,:,-1] / z[:,:,-1].max() * alpha

    z[:,:,-1] = z[:,:,-1].clip(min=0, max=alphalim)
    
    

    im2 = None
    im3 = None
    ymax_full = ymax
    ymin_full = ymin
    xmax_full = xmax
    xmin_full = xmin
    if ymin < lim[0]:
        zz = z.copy().clip(max=alphalim, min=0)
        zz = (zz + z[:1].clip(max=alphalim, min=0)).clip(max=alphalim, min=0)
        zz[:,:,:3] = z[:,:,:3]
        im2 = ax.imshow(zz.clip(max=alphalim, min=0), aspect='auto', extent=[xmin, xmax, ymin, lim[0]],
                    origin='lower', zorder=zorder)
        ymin = lim[0]
    if xmax > xmaxlim:
        im3 = ax.imshow([[[rgb[0], rgb[1], rgb[2], alphalim]]], aspect='auto', extent=[xmaxlim-0.001, xmax, ymin, ymax],
                    origin='lower', zorder=zorder)
        xmax = xmaxlim


    if ymax > lim[1]:
        zz = z.copy().clip(max=alphalim, min=0)
        zz = (zz + z[-1:].clip(max=alphalim, min=0)).clip(max=alphalim, min=0)
        zz[:,:,:3] = z[:,:,:3]
        im3 = ax.imshow(zz.clip(max=alphalim, min=0), aspect='auto', extent=[xmin, xmax, ymin, ymax],
                    origin='lower', zorder=zorder)
        ymax = lim[1]
    if xmin < -xmaxlim:
        im2 = ax.imshow([[[rgb[0], rgb[1], rgb[2], alphalim]]], aspect='auto', extent=[xmin, -xmaxlim+0.001, ymin, ymax],
                    origin='lower', zorder=zorder)
        xmin = -xmaxlim

    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    if direction == 'down':
        xy = np.vstack([[xmin_full, ymin_full], xy, [xmax_full, ymin_full], [xmin_full, ymin_full]])
    else: # direction=='up'
        xy = np.vstack([[xmin_full, ymax_full], xy, [xmax_full, ymax_full], [xmin_full, ymax_full]])        
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)
    if im2 is not None:
        im2.set_clip_path(clip_path)
    if im3 is not None:
        im3.set_clip_path(clip_path)

    ax.autoscale(True)
    return line, im

    

df_occ['emp_size'] = emp_size = 30 +  0.0003*df_occ["TOT_EMP"]
criteria_neigh_pos_before=0.025
criteria_neigh_neg_before=-0.025
criteria_shock_before = 0.03

criteria_neigh_pos_after=0.00
criteria_neigh_neg_after=-0.02
criteria_shock_after = -0.035

#####
# Plotting choices
#####
bool_perm = df_occ['Permanent_boost_r0.01'] == 1
bool_phaseout = df_occ['Phase_out_r0.01'] == 1
bool_temp = df_occ['Temporary_boost_r0.01'] == 1
bool_late = df_occ['Late_boost_r0.01'] == 1
bool_out_before = (bool_phaseout)|(bool_late)
bool_in_before = (bool_temp)|( bool_perm)
bool_out_after = (bool_temp)|(bool_phaseout)
bool_in_after = ( bool_perm)|(bool_late)

df_occ_in_pos = df_occ[bool_in_before].reset_index(drop=True)
df_occ_out_neg = df_occ[bool_out_before].reset_index(drop=True)

df_occ_in_pos_after = df_occ[bool_in_after].reset_index(drop=True)
df_occ_out_neg_after = df_occ[bool_out_after ].reset_index(drop=True)

In [ ]:
df_occ_in_pos.columns

In [ ]:

df_selected_before_pos = selected_occ(df_occ_in_pos, shock=shock_before_name + '_norm',
                        neighbors_in=neigh_in_bef,\
                        neighbors_out=neigh_out_bef,
                        shock_bool='pos', criteria_shock=criteria_shock_before,\
                        criteria_neigh_pos=criteria_neigh_pos_before, \
                        criteria_neigh_neg=criteria_neigh_neg_before)

df_selected_before_neg = selected_occ(df_occ_out_neg , shock=shock_before_name + '_norm',
                        neighbors_in=neigh_in_bef,\
                        neighbors_out=neigh_out_bef,
                        shock_bool='neg', criteria_shock=criteria_shock_before,\
                        criteria_neigh_pos=criteria_neigh_pos_before, \
                        criteria_neigh_neg=criteria_neigh_neg_before)


In [ ]:
def circle_selected(sc, ax, df, occupations_with_circle = circle_occs,
                    occupation_codes = circle_occs_code, shorter=False):
    for occ, code in list(zip(occupations_with_circle, circle_occs_code)):
        print(occ)
        i = df[df.OCC_TITLE == occ].index.tolist()
        if len(i) == 0:
            continue
        else:
            i = i[0]
        print(i)
        if len(sc.get_offsets().data) > 2:
            x, y =  sc.get_offsets().data[i]
        elif len(sc.get_offsets().data) == 2:
            if i == 41:
                i = 0
            else:
                i = 1
            x, y =  sc.get_offsets().data[i]
        else:
            x, y =  sc.get_offsets().data[0]
        ax.plot(x,y,marker='o',ms=16,mfc="None",mec='r', linewidth=4)
        ax.annotate(code, (x, y+0.0005 if shorter else y+0.0008), horizontalalignment='center',
                    verticalalignment='bottom', c='r', fontsize=15)

df_selected_after_pos = selected_occ(df_occ_in_pos_after, shock=shock_after_name + '_norm',
                        neighbors_in=neigh_in_af,\
                        neighbors_out=neigh_out_af,
                        shock_bool='pos', criteria_shock=criteria_shock_after,\
                        criteria_neigh_pos=0.025, \
                        criteria_neigh_neg=criteria_neigh_neg_after)

df_selected_after_neg = selected_occ(df_occ_out_neg_after, shock=shock_after_name + '_norm',
                        neighbors_in=neigh_in_af,\
                        neighbors_out=neigh_out_af,
                        shock_bool='neg', criteria_shock=criteria_shock_after,\
                        criteria_neigh_pos=criteria_neigh_pos_after, \
                        criteria_neigh_neg=criteria_neigh_neg_after)


#### 
# Plots
####

color_in = 'k'#'g'
color_out = 'k'#'r'
symlogthresh_y = 0.05
if neigh_in_bef  == 'frac_updown_inbefore':
    symlogthresh_y = 0.99
symlogthresh_x = 0.05

color_quarter_text = "purple"
color_quarter = "k"
egcolors = 'grey'
lnwidth = 0.3
 

####
# Plot Before
####

######
labelticks_x = [-0.2, 0.0, 0.2, 0.4, 0.6, 0.8]
# labels y
    
labelticks_y = [-0.05, -0.02, -0.01, 0, 0.01, 0.02, 0.03, 0.05]
if network_type == 'cc':
    labelticks_y = [-0.05, -0.02, -0.01, 0, 0.01, 0.02, 0.03, 0.05]
if network_type == 'tasksim':
    labelticks_y = [-0.025, 0, 0.025]
# choose which ones to label and add percentage
ticks_x = np.linspace(-0.2, 1, 13)
ticks_x = np.round(ticks_x , 5)
new_labels_x = [str(t*100) + "%" if t in labelticks_x \
    else "" for t in ticks_x]
# choose which ones to label and add percentage
ticks_y = np.linspace(-0.01, 0.03, 9)
if network_type == 'cc':
    ticks_y = np.linspace(-0.02, 0.03, 11)
if network_type == 'tasksim':
    ticks_y = np.linspace(-0.025, 0.025, 10)
ticks_y = np.round(ticks_y , 5)
new_labels_y = [str(t*100) + "%" if t in labelticks_y \
    else "" for t in ticks_y]

labelticks_zoom_x = [-0.05, -0.025, 0, 0.025, 0.05]
labelticks_zoom_y = [0.00, 0.01, 0.02]
ticks_zoom_x = np.linspace(-0.05, 0.05, 5)
ticks_zoom_x = np.round(ticks_zoom_x , 5)
new_labels_zoom_x = [str(t*100) + "%" if t in labelticks_zoom_x \
    else "" for t in ticks_zoom_x]
ticks_zoom_y = np.linspace(0.00, 0.025, 6)
ticks_zoom_y = np.round(ticks_zoom_y , 5)
new_labels_zoom_y = [str(t*100) + "%" if t in labelticks_zoom_y \
    else "" for t in ticks_zoom_y]

#####

max_wage = max(df_occ['A_MEAN']) 
min_wage = min(df_occ['A_MEAN']) 
norm = matplotlib.colors.LogNorm(vmin=min_wage, vmax=max_wage)

label_y1 = 'In-neighbors average percentage shock (\u25CF)'
label_y2 = 'Out-neighbors average percentage shock (\u25A0)'

fig = plt.figure(figsize=(10.5, 8))
ax1 = plt.subplot()
ax1_twin = ax1.twinx()
ax1.tick_params(labelsize=16)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax1_twin.tick_params(labelsize=16)
ax1_twin.spines['left'].set_visible(False)
ax1_twin.spines['top'].set_visible(False)
ax1_twin.spines['bottom'].set_visible(False)


ax1.axhline(0, c=color_quarter, linestyle="--", linewidth=0.2)
# ax2.axhline(0, c=color_quarter, linestyle="--", linewidth=0.2)
ax1.set_ylim([-0.01, 0.05])
ax1_twin.set_ylim([-0.01, 0.05])
if network_type == 'cc':
    ax1.set_ylim([-0.01, 0.05])
    ax1_twin.set_ylim([-0.01, 0.05])
if network_type == 'tasksim':
    ax1.set_ylim([-0.025, 0.025])
    ax1_twin.set_ylim([-0.025, 0.025])
if neigh_in_bef  == 'frac_updown_inbefore':
    ax1.set_ylim([-1, 1])
    ax1_twin.set_ylim([-1, 1])
ax1.set_xlim([-0.15, 0.85])
ax1_twin.set_xlim([-0.15, 0.85])

ax1.set_yticks(ticks_y)
ax1_twin.set_yticks(ticks_y)
ax1.set_yticklabels(new_labels_y)
ax1_twin.set_yticklabels(new_labels_y)
ax1.set_xticks(ticks_x)
ax1.set_xticklabels(new_labels_x)


axins = ax1_twin.inset_axes([0.35, 0.55, 0.55, 0.8])

x1, x2, y1, y2 =  -0.06, 0.05, -0.002, 0.025
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

axins_twin = axins.twinx()
axins_twin.set_xlim(x1, x2)
axins_twin.set_ylim(y1, y2)

axins.set_yticks(ticks_zoom_y)
axins.set_yticklabels(new_labels_zoom_y)
axins.set_xticks(ticks_zoom_x)
axins.set_xticklabels(new_labels_zoom_x)

axins_twin.set_yticks(ticks_zoom_y)
axins_twin.set_yticklabels(new_labels_zoom_y)
axins_twin.set_xticks(ticks_zoom_x)
axins_twin.set_xticklabels(new_labels_zoom_x)

axins.tick_params(labelsize=16)
axins_twin.tick_params(labelsize=16)

rect, lines = ax1.indicate_inset_zoom(axins)

axins.axhline(0, c=color_quarter, linestyle="--", linewidth=0.2)


# positive shocks before
sc_twin = ax1_twin.scatter(df_occ_in_pos[shock_before_name + '_norm']\
    ,df_occ_in_pos[neigh_in_bef],
    c=df_occ_in_pos['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_in_pos['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth)

# negative shocks before
sc = ax1.scatter(df_occ_out_neg[shock_before_name + '_norm']\
    ,df_occ_out_neg[neigh_out_bef]\
    , c=df_occ_out_neg['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_out_neg['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth, marker='s')

sc_axins = axins.scatter(df_occ_out_neg[shock_before_name + '_norm']\
    ,df_occ_out_neg[neigh_out_bef]\
    , c=df_occ_out_neg['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_out_neg['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth, marker='s')

sc_axins_twin = axins_twin.scatter(df_occ_in_pos[shock_before_name + '_norm']\
    ,df_occ_in_pos[neigh_in_bef],
    c=df_occ_in_pos['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_in_pos['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth)

ax1_twin.set_ylabel(label_y1, color=color_in,fontsize=16)
ax1.set_ylabel(label_y2, color=color_out, fontsize=16)

ax1.set_xlabel("Percentage change in demand " + str(start_year) + "-" \
    +str(mark_year), \
    fontsize=16)

fig.subplots_adjust(bottom=0.2)
cbar_ax = fig.add_axes([0.124, 0.1, 0.78, 0.02])
cbar = plt.colorbar(sc, ticks=[2e4, 3e4, 4e4, 6e4, 1.0e5, 2e5], pad=0.1023, 
                        orientation='horizontal',
                        aspect=30, cax=cbar_ax)
cbar.set_label('Median wage (2018-USD annually)', fontsize=14)

# set colorbar limits
cbar.mappable.set_clim(20000, 150000)

import matplotlib.ticker as ticker
# set colorbar ticks format to be in thousands
cbar.ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))


if circle_selected_bool:
    circle_selected(sc, ax1, df_occ_out_neg)
    circle_selected(sc_twin,ax1_twin, df_occ_in_pos)
    circle_selected(sc_axins, axins, df_occ_out_neg)
    circle_selected(sc_axins_twin, axins_twin, df_occ_in_pos)
    # circle_selected(sc2,ax2, df_occ_in_pos[df_occ_in_pos['OCC_TITLE'].str.contains('Wind|Solar')])

# make background of sc_axins and sc_axins_twin white


if background_bool:
    background_color(sc, ax1, [-0.85, 0.85], [-0.03, 0.03])
    background_color(sc_twin,ax1_twin, [-0.85, 0.85], [-0.03, 0.03])
    background_color(sc_axins, axins, [-0.06, 0.05], [-0.03, 0.03], identity=True)
    background_color(sc_axins_twin, axins_twin, [-0.06, 0.05], [-0.03, 0.03], identity=True)
    background_text(sc, ax1, -87)
    background_text(sc_twin, ax1_twin, -87)
    background_text(sc_axins_twin, axins_twin, -75, zoombox=True)
    vertical_line_with_text(sc, ax1)
    vertical_line_with_text(sc_twin, ax1_twin)
    vertical_line_with_text(sc_axins, axins)
    vertical_line_with_text(sc_axins_twin, axins_twin)
if network_type == 'cc':
    ax1.set_ylim([-0.01, 0.05])
    ax1_twin.set_ylim([-0.01, 0.05])   

ax1.set_xlim([-0.2, 0.85])
ax1_twin.set_xlim([-0.2, 0.85])
ax1.set_ylim([-0.005, 0.03])
ax1_twin.set_ylim([-0.005, 0.03])

x1, x2, y1, y2 =  -0.06, 0.05, -0.002, 0.025
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

axins_twin.set_xlim(x1, x2)
axins_twin.set_ylim(y1, y2)
    
if annotate_plot_bool:
    annotate_plot(sc,ax1, df_selected_before_neg)
    annotate_plot(sc_twin,ax1_twin, df_selected_before_pos)
    annotate_plot(sc_axins,axins, df_selected_before_neg)
    annotate_plot(sc_axins_twin,axins_twin, df_selected_before_pos)    
    plt.savefig(path_fig + fig_out_name + "_annotated.svg")
    plt.savefig(path_fig + fig_out_name +  "_annotated.png")
    plt.show()
else:
    plt.savefig(path_fig + fig_out_name + ".png")
    plt.show()








####
# Plot After
####

#annotate_plot_bool = False #True

######
labelticks_x = [-0.05, 0.0, 0.05, 0.1, 0.15, 0.2]
# labels y

labelticks_y = [-0.03, -0.02, -0.01, 0]
# choose which ones to label and add percentage
ticks_x = np.linspace(-0.05, 0.2, 6)
ticks_x = np.round(ticks_x , 5)
new_labels_x = [str(t*100) + "%" if t in labelticks_x \
    else "" for t in ticks_x]
# choose which ones to label and add percentage
ticks_y = np.linspace(-0.03, 0.005, 8)
ticks_y = np.round(ticks_y , 5)
new_labels_y = [str(t*100) + "%" if t in labelticks_y \
    else "" for t in ticks_y]


labelticks_zoom_x = [-0.03, -0.02, -0.01, 0]
labelticks_zoom_y = [-0.01, -0.005, 0]
ticks_zoom_x = np.linspace(-0.03, 0.0, 4)
ticks_zoom_x = np.round(ticks_zoom_x , 5)
new_labels_zoom_x = [str(t*100) + "%" if t in labelticks_zoom_x \
    else "" for t in ticks_zoom_x]
ticks_zoom_y = np.linspace(-0.01, 0.0, 3)
ticks_zoom_y = np.round(ticks_zoom_y , 5)
new_labels_zoom_y = [str(t*100) + "%" if t in labelticks_zoom_y \
    else "" for t in ticks_zoom_y]

#####

max_wage = max(df_occ['A_MEAN']) 
min_wage = min(df_occ['A_MEAN']) 
norm = matplotlib.colors.LogNorm(vmin=min_wage, vmax=max_wage)

label_y1 = 'In-neighbors average percentage shock (\u25CF)'
label_y2 = 'Out-neighbors average percentage shock (\u25A0)'

fig = plt.figure(figsize=(10.5, 12))
ax1 = plt.subplot()
ax1_twin = ax1.twinx()

ax1.tick_params(labelsize=16)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax1_twin.tick_params(labelsize=16)
ax1_twin.spines['left'].set_visible(False)
ax1_twin.spines['top'].set_visible(False)
ax1_twin.spines['bottom'].set_visible(False)

ax1.axhline(0, c=color_quarter, linestyle="--", linewidth=0.2)
ax1.set_ylim([-0.03, 0.005])
ax1_twin.set_ylim([-0.03, 0.005])
if network_type == 'cc':
    ax1.set_ylim([-0.03, 0.005])
    ax1_twin.set_ylim([-0.03, 0.005])
if network_type == 'tasksim':
    ax1.set_ylim([-0.025, 0.025])
    ax1_twin.set_ylim([-0.025, 0.025])
if neigh_in_bef  == 'frac_updown_inbefore':
    ax1.set_ylim([-1, 1])
    ax1_twin.set_ylim([-1, 1])
ax1.set_xlim([-0.06, 0.2])
ax1_twin.set_xlim([-0.06, 0.2])

ax1.set_yticks(ticks_y)
ax1_twin.set_yticks(ticks_y)
ax1.set_yticklabels(new_labels_y)
ax1_twin.set_yticklabels(new_labels_y)
ax1.set_xticks(ticks_x)
ax1.set_xticklabels(new_labels_x)

axins = ax1_twin.inset_axes([0.45, 0.12, 0.45, 0.45])

x1, x2, y1, y2 =  -0.03, 0.0008, -0.01, 0.0003
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

axins_twin = axins.twinx()
axins_twin.set_xlim(x1, x2)
axins_twin.set_ylim(y1, y2)

axins.set_yticks(ticks_zoom_y)
axins.set_yticklabels(new_labels_zoom_y)
axins.set_xticks(ticks_zoom_x)
axins.set_xticklabels(new_labels_zoom_x)

axins_twin.set_yticks(ticks_zoom_y)
axins_twin.set_yticklabels(new_labels_zoom_y)
axins_twin.set_xticks(ticks_zoom_x)
axins_twin.set_xticklabels(new_labels_zoom_x)

axins.tick_params(labelsize=16)
axins_twin.tick_params(labelsize=16)

rect, lines = ax1.indicate_inset_zoom(axins)

axins.axhline(0, c=color_quarter, linestyle="--", linewidth=0.2)

# positive shocks before
sc_twin = ax1_twin.scatter(df_occ_in_pos_after[shock_after_name + '_norm']\
    ,df_occ_in_pos_after[neigh_in_af],
    c=df_occ_in_pos_after['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_in_pos_after['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth )

# negative shocks before
sc = ax1.scatter(df_occ_out_neg_after[shock_after_name + '_norm']\
    ,df_occ_out_neg_after[neigh_out_af]\
    , c=df_occ_out_neg_after['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_out_neg_after['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth, marker='s')

sc_axins = axins.scatter(df_occ_out_neg_after[shock_after_name + '_norm']\
    ,df_occ_out_neg_after[neigh_out_af]\
    , c=df_occ_out_neg_after['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_out_neg_after['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth, marker='s')

sc_axins_twin = axins_twin.scatter(df_occ_in_pos_after[shock_after_name + '_norm']\
    ,df_occ_in_pos_after[neigh_in_af],
    c=df_occ_in_pos_after['A_MEAN'],cmap="viridis", \
        norm=norm, alpha=0.8, \
    s=df_occ_in_pos_after['emp_size'], edgecolors=egcolors,\
        linewidths=lnwidth)


ax1_twin.set_ylabel(label_y1, color=color_in,fontsize=16)
ax1.set_ylabel(label_y2, color=color_out, fontsize=16)

ax1.set_xlabel("Percentage change in demand " + str(mark_year) + "-" \
    +str(end_year), \
    fontsize=16)

fig.subplots_adjust(bottom=0.2)
cbar_ax = fig.add_axes([0.124, 0.1, 0.78, 0.02])

cbar = plt.colorbar(sc, ticks=[2e4, 3e4, 4e4, 6e4, 1.0e5, 2e5], pad=0.1023, 
                    orientation='horizontal',
                    aspect=30, cax=cbar_ax)
cbar.set_label('Median wage (2018-USD annually)', fontsize=14)

# set colorbar limits
cbar.mappable.set_clim(20000, 150000)

import matplotlib.ticker as ticker
# set colorbar ticks format to be in thousands
cbar.ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

if circle_selected_bool:
    circle_selected(sc, ax1, df_occ_out_neg_after, shorter=True)
    circle_selected(sc_twin,ax1_twin, df_occ_in_pos_after, shorter=True)
    circle_selected(sc_axins, axins, df_occ_out_neg_after, shorter=True)
    circle_selected(sc_axins_twin, axins_twin, df_occ_in_pos_after, shorter=True)
    # circle_selected(sc2,ax2, df_occ_in_pos_after[df_occ_in_pos_after['OCC_TITLE'].str.contains('Wind')])
    
if background_bool:
    background_color(sc, ax1, [-0.06, 0.2], [-0.03, 0.03])
    background_color(sc_twin,ax1_twin, [-0.06, 0.2], [-0.03, 0.03])
    background_color(sc_axins, axins, [-0.03, 0.0], [-0.03, 0.03], identity=True)
    background_color(sc_axins_twin, axins_twin, [-0.03, 0.0], [-0.03, 0.3], identity=True)
    # background_color(sc2, ax2, [0.17, 0.20], [-0.03, 0.01], identity=False)
    background_text(sc, ax1, -82, identity='up')
    background_text(sc_twin, ax1_twin, -82, identity = 'up')
    background_text(sc_axins, axins, -73, zoombox=True, identity='up')
    background_text(sc_axins_twin, axins_twin, -73, zoombox=True, identity='up')
    vertical_line_with_text(sc, ax1, identity='up')
    vertical_line_with_text(sc_twin, ax1_twin, identity='up')
    vertical_line_with_text(sc_axins, axins, identity='up')
    vertical_line_with_text(sc_axins_twin, axins_twin, identity='up')
if network_type == 'cc':
    ax1.set_ylim([-0.03, 0.005])
    ax1_twin.set_ylim([-0.03, 0.005])

ax1.set_xlim([-0.06, 0.2])
ax1_twin.set_xlim([-0.06, 0.2])

x1, x2, y1, y2 =  -0.03, 0.0008, -0.01, 0.0003
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

axins_twin.set_xlim(x1, x2)
axins_twin.set_ylim(y1, y2)
    
if annotate_plot_bool:
    ### NOTE proble, with double axis and one sc
    annotate_plot(sc,ax1, df_selected_after_neg)
    annotate_plot(sc_twin,ax1_twin, df_selected_after_pos)
    annotate_plot(sc_axins,axins, df_selected_after_neg)
    annotate_plot(sc_axins_twin,axins_twin, df_selected_after_pos)
    plt.savefig(path_fig + fig_out_name_after + "_annotated.svg")
    plt.savefig(path_fig + fig_out_name_after +  "_annotated.png")
    plt.show()
else:
    plt.savefig(path_fig + fig_out_name_after + ".png")
    plt.show()